# STAC Catalog Setup
CFolkers
Geospatial Services 
2024 02 12

modified from https://github.com/stac-utils/pystac/blob/8079dd3c0cbe8f6f9e48f499ea90f6a5798eaeab/docs/tutorials/how-to-create-stac-catalogs.ipynb

In [2]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os
import pystac 

loading dot env...


In [6]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET


In [11]:
#list .laz objects in bucket
object_key="STAC_LiDAR/PointClouds/"
laz_objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=object_key, StartAfter=object_key)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        laz_objects.append(obj['Key'])
        print(obj['Key'])
else:
    print("No objects found in the bucket.")
    

STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_3_4_2_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_3_4_4_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_4_1_3_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_4_1_4_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_4_3_1_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_4_3_2_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_4_3_3_xyes_12_utm10_2018.laz
STAC_LiDAR/PointClouds/bc_092o018_4_3_4_xyes_12_utm10_2018.laz


In [15]:
#Create URL to access .laz file 
url_dict={}
for laz in laz_objects:
    presigned_url=s3_client.generate_presigned_url('get_object',
                                        Params={'Bucket': bucket, 'Key': laz},
                                        ExpiresIn=3600)  # Expiration time in seconds (e.g., 1 hour)
    # print(presigned_url)
    url_dict[laz]=presigned_url

{'STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz': 'https://nrs.objectstore.gov.bc.ca:443/rczimv/STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz?AWSAccessKeyId=nr-geobc-data-test&Signature=ePLYK1oppeWsoYeivX7Irpj0kRU%3D&Expires=1707771788', 'STAC_LiDAR/PointClouds/bc_092o018_3_4_2_xyes_12_utm10_2018.laz': 'https://nrs.objectstore.gov.bc.ca:443/rczimv/STAC_LiDAR/PointClouds/bc_092o018_3_4_2_xyes_12_utm10_2018.laz?AWSAccessKeyId=nr-geobc-data-test&Signature=UwSAH03laBaZTPGXtksbkISlob8%3D&Expires=1707771788', 'STAC_LiDAR/PointClouds/bc_092o018_3_4_4_xyes_12_utm10_2018.laz': 'https://nrs.objectstore.gov.bc.ca:443/rczimv/STAC_LiDAR/PointClouds/bc_092o018_3_4_4_xyes_12_utm10_2018.laz?AWSAccessKeyId=nr-geobc-data-test&Signature=KLUbqk1y68CTsykEKeYWFw8cmAY%3D&Expires=1707771788', 'STAC_LiDAR/PointClouds/bc_092o018_4_1_3_xyes_12_utm10_2018.laz': 'https://nrs.objectstore.gov.bc.ca:443/rczimv/STAC_LiDAR/PointClouds/bc_092o018_4_1_3_xyes_12_utm10_2018.laz?AWSAccessKe

In [3]:
?pystac.Catalog

Init signature:
pystac.Catalog(
    id: 'str',
    description: 'str',
    title: 'str | None' = None,
    stac_extensions: 'list[str] | None' = None,
    extra_fields: 'dict[str, Any] | None' = None,
    href: 'str | None' = None,
    catalog_type: 'CatalogType' = ABSOLUTE_PUBLISHED,
)
Docstring:     
A PySTAC Catalog represents a STAC catalog in memory.

A Catalog is a :class:`~pystac.STACObject` that may contain children,
which are instances of :class:`~pystac.Catalog` or :class:`~pystac.Collection`,
as well as :class:`~pystac.Item` s.

Args:
    id : Identifier for the catalog. Must be unique within the STAC.
    description : Detailed multi-line description to fully explain the catalog.
        `CommonMark 0.29 syntax <https://commonmark.org/>`_ MAY be used for rich
        text representation.
    title : Optional short descriptive one-line title for the catalog.
    stac_extensions : Optional list of extensions the Catalog implements.
    href : Optional HREF for this catalog, w

In [4]:
catalog = pystac.Catalog(id="lidar-test", description="Test catalog for the potential use of STAC to access open LiDAR Data")